In [1]:
import pandas as pd

# 定義轉換函數，將季度EPS轉換為月度EPS
def process_quarterly_eps_sheet(sheet_df):
    month_data = []

    # 遍歷每行數據，進行季度到月度的轉換
    for i in range(len(sheet_df) - 1):
        current_quarter_eps = sheet_df.iloc[i]['每股盈餘']
        next_quarter_eps = sheet_df.iloc[i + 1]['每股盈餘']

        # 解析"年/月"欄位，提取年份和月份
        year_month = sheet_df.iloc[i]['年/月']
        year, month = map(int, year_month.split('/'))

        # 計算每月的線性插值
        month_1_eps = current_quarter_eps
        month_2_eps = current_quarter_eps + (next_quarter_eps - current_quarter_eps) / 3
        month_3_eps = current_quarter_eps + 2 * (next_quarter_eps - current_quarter_eps) / 3

        # 根據月份確定對應的季度月份（假設季末在3月、6月、9月和12月）
        if month == 3:
            months = ['Jan', 'Feb', 'Mar']
        elif month == 6:
            months = ['Apr', 'May', 'Jun']
        elif month == 9:
            months = ['Jul', 'Aug', 'Sep']
        elif month == 12:
            months = ['Oct', 'Nov', 'Dec']

        # 將每個月的資料附加到列表
        month_data.append({'Year': year, 'Month': months[0], 'EPS': month_1_eps})
        month_data.append({'Year': year, 'Month': months[1], 'EPS': month_2_eps})
        month_data.append({'Year': year, 'Month': months[2], 'EPS': month_3_eps})

    # 將結果轉換為DataFrame
    monthly_eps_df = pd.DataFrame(month_data)

    return monthly_eps_df

In [2]:
# 讀取Excel文件
file_path = '文創每股盈餘.xlsx'  # 請將此路徑替換為你的實際檔案路徑
excel_file = pd.ExcelFile(file_path)

# 準備保存每個工作表處理後的數據
monthly_data = {}

# 處理每個工作表
for sheet in excel_file.sheet_names:
    quarterly_eps_df = pd.read_excel(file_path, sheet_name=sheet)
    monthly_eps_df = process_quarterly_eps_sheet(quarterly_eps_df)
    monthly_data[sheet] = monthly_eps_df

# 將結果保存為新的Excel文件
output_file_path = 'expanded_monthly_eps_5.xlsx'
with pd.ExcelWriter(output_file_path, engine='xlsxwriter') as writer:
    for sheet, data in monthly_data.items():
        data.to_excel(writer, sheet_name=sheet, index=False)

print(f"月度EPS資料已保存至 {output_file_path}")


月度EPS資料已保存至 expanded_monthly_eps_5.xlsx
